This is notebook gives a quick overview of this WaveNet implementation, i.e. creating the model and the data set, training the model and generating samples from it.

In [1]:
from model_logging import *
import torch
from wavenet_model import *
from audio_data import WavenetDataset
from wavenet_training import *
#from model_logging import *

## Model
This is an implementation of WaveNet as it was described in the original paper (https://arxiv.org/abs/1609.03499). Each layer looks like this:

```
            |----------------------------------------|      *residual*
            |                                        |
            |    |-- conv -- tanh --|                |
 -> dilate -|----|                  * ----|-- 1x1 -- + -->  *input*
                 |-- conv -- sigm --|     |
                                         1x1
                                          |
 ---------------------------------------> + ------------->  *skip*
```

Each layer dilates the input by a factor of two. After each block the dilation is reset and start from one. You can define the number of layers in each block (``layers``) and the number of blocks (``blocks``). The blocks are followed by two 1x1 convolutions and a softmax output function.
Because of the dilation operation, the independent output for multiple successive samples can be calculated efficiently. With ``output_length``, you can define the number these outputs. Empirically, it seems that a large number of skip channels is required.

In [2]:
# initialize cuda option
dtype = torch.FloatTensor # data type
ltype = torch.LongTensor # label type

use_cuda = torch.cuda.is_available()
#use_cuda = False
if use_cuda:
    print('use gpu')
    dtype = torch.cuda.FloatTensor
    ltype = torch.cuda.LongTensor

use gpu


In [3]:
model = WaveNetModel(layers=10,
                     blocks=3,
                     dilation_channels=32,
                     residual_channels=32,
                     skip_channels=1024,
                     end_channels=512, 
                     output_length=16,
                     dtype=dtype, 
                     bias=True)
# model = load_latest_model_from('snapshots', use_cuda=use_cuda)

print('model: ', model.cuda())
print('receptive field: ', model.receptive_field)
print('parameter count: ', model.parameter_count())

model:  WaveNetModel (
  (filter_convs): ModuleList (
    (0): Conv1d(32, 32, kernel_size=(2,), stride=(1,))
    (1): Conv1d(32, 32, kernel_size=(2,), stride=(1,))
    (2): Conv1d(32, 32, kernel_size=(2,), stride=(1,))
    (3): Conv1d(32, 32, kernel_size=(2,), stride=(1,))
    (4): Conv1d(32, 32, kernel_size=(2,), stride=(1,))
    (5): Conv1d(32, 32, kernel_size=(2,), stride=(1,))
    (6): Conv1d(32, 32, kernel_size=(2,), stride=(1,))
    (7): Conv1d(32, 32, kernel_size=(2,), stride=(1,))
    (8): Conv1d(32, 32, kernel_size=(2,), stride=(1,))
    (9): Conv1d(32, 32, kernel_size=(2,), stride=(1,))
    (10): Conv1d(32, 32, kernel_size=(2,), stride=(1,))
    (11): Conv1d(32, 32, kernel_size=(2,), stride=(1,))
    (12): Conv1d(32, 32, kernel_size=(2,), stride=(1,))
    (13): Conv1d(32, 32, kernel_size=(2,), stride=(1,))
    (14): Conv1d(32, 32, kernel_size=(2,), stride=(1,))
    (15): Conv1d(32, 32, kernel_size=(2,), stride=(1,))
    (16): Conv1d(32, 32, kernel_size=(2,), stride=(1,))
    

## Data Set
To create the data set, you have to specify a path to a data set file. If this file already exists it will be used, if not it will be generated. If you want to generate the data set file (a ``.npz`` file), you have to specify the directory (``file_location``) in which all the audio files you want to use are located. The attribute ``target_length`` specifies the number of successive samples are used as a target and corresponds to the output length of the model. The ``item_length`` defines the number of samples in each item of the dataset and should always be ``model.receptive_field + model.output_length - 1``.

```
          |----receptive_field----|
                                |--output_length--|
example:  | | | | | | | | | | | | | | | | | | | | |
target:                           | | | | | | | | | |  
```
To create a test set, you should define a ``test_stride``. Then each ``test_stride``th item will be assigned to the test set.

In [4]:
data = WavenetDataset(dataset_file='train_samples/bach_chaconne/dataset.npz',
                      item_length=model.receptive_field + model.output_length - 1,
                      target_length=model.output_length,
                      file_location='train_samples/bach_chaconne',
                      test_stride=500)
print('the dataset has ' + str(len(data)) + ' items')

one hot input
the dataset has 598277 items


## Training and Logging
This implementation supports logging with TensorBoard (you need to have TensorFlow installed). You can even generate audio samples from the current snapshot of the model during training. This will happen in a background thread on the cpu, so it will not interfere with the actual training but will be rather slow. If you don't have TensorFlow, you can use the standard logger that will print out to the console.
The trainer uses Adam as default optimizer.

In [9]:
def generate_and_log_samples(step):
    sample_length=32000
    gen_model = load_latest_model_from('snapshots', use_cuda=False)
    print("start generating...")
    samples = generate_audio(gen_model,
                             length=sample_length,
                             temperatures=[0.5])
    tf_samples = tf.convert_to_tensor(samples, dtype=tf.float32)
    logger.audio_summary('temperature_0.5', tf_samples, step, sr=16000)

    samples = generate_audio(gen_model,
                             length=sample_length,
                             temperatures=[1.])
    tf_samples = tf.convert_to_tensor(samples, dtype=tf.float32)
    logger.audio_summary('temperature_1.0', tf_samples, step, sr=16000)
    print("audio clips generated")


#logger = TensorboardLogger(log_interval=200,
 #                          validation_interval=400,
  #                         generate_interval=1000,
   #                        generate_function=generate_and_log_samples,
    #                       log_dir="logs/chaconne_model")

logger = Logger(log_interval=20,
                 validation_interval=400,
                 generate_interval=1000)

In [ ]:
trainer = WavenetTrainer(model=model,
                         dataset=data,
                         lr=0.001,
                         snapshot_path='snapshots',
                         snapshot_name='chaconne_model',
                         snapshot_interval=1000,
                         logger=logger,
                         dtype=dtype,
                         ltype=ltype)

print('start training...')
trainer.train(batch_size=8,
              epochs=10)

start training...
epoch 0
loss at step 20: 5.162780165672302
loss at step 40: 5.089954590797424
loss at step 60: 5.054309391975403
loss at step 80: 4.960115432739258
one training step does take approximately 0.5236522936820984 seconds)
loss at step 100: 4.897372150421143
loss at step 120: 4.790813255310058
loss at step 140: 4.636266350746155
loss at step 160: 4.484929537773132
loss at step 180: 4.268103325366974
loss at step 200: 4.15226628780365
loss at step 220: 4.086933124065399
loss at step 240: 4.068137288093567
loss at step 260: 4.022809708118439
loss at step 280: 4.026028096675873
loss at step 300: 3.888850724697113
loss at step 320: 3.8549909591674805
loss at step 340: 3.834302806854248
loss at step 360: 3.79020094871521
loss at step 380: 3.7681735157966614
loss at step 400: 3.810551917552948
validation loss: 4.034210497538249
validation accuracy: 6.787353923205343%
loss at step 420: 3.756807065010071
loss at step 440: 3.706703209877014
loss at step 460: 3.831369996070862
loss 

validation loss: 3.4052259667714435
validation accuracy: 9.24457429048414%
loss at step 4020: 3.1789023518562316
loss at step 4040: 3.2641913294792175
loss at step 4060: 3.177927255630493
loss at step 4080: 3.2050485610961914
loss at step 4100: 3.2564444184303283
loss at step 4120: 3.162487733364105
loss at step 4140: 3.192570388317108
loss at step 4160: 3.2074580430984496
loss at step 4180: 3.2347574710845945
loss at step 4200: 3.1917534470558167
loss at step 4220: 3.2261887431144713
loss at step 4240: 3.1985196232795716
loss at step 4260: 3.2420174837112428
loss at step 4280: 3.220473349094391
loss at step 4300: 3.1718783020973205
loss at step 4320: 3.1213488936424256
loss at step 4340: 3.2046050906181334
loss at step 4360: 3.2713900446891784
loss at step 4380: 3.225289046764374
loss at step 4400: 3.1465413928031922
validation loss: 3.3603880818684897
validation accuracy: 9.307178631051753%
loss at step 4420: 3.1918266654014587
loss at step 4440: 3.1917948484420777
loss at step 4460:

loss at step 8000: 3.114830756187439
validation loss: 3.235477124849955
validation accuracy: 10.079298831385643%
loss at step 8020: 3.128624749183655
loss at step 8040: 3.111064302921295
loss at step 8060: 3.221082878112793
loss at step 8080: 3.1077369809150697
loss at step 8100: 3.012094271183014
loss at step 8120: 3.03189754486084
loss at step 8140: 3.1080143094062804
loss at step 8160: 3.0648762583732605
loss at step 8180: 3.0929367184638976
loss at step 8200: 3.075855779647827
loss at step 8220: 3.095041048526764
loss at step 8240: 3.0323700189590452
loss at step 8260: 3.1083167552948
loss at step 8280: 3.100862979888916
loss at step 8300: 3.156915545463562
loss at step 8320: 3.057618248462677
loss at step 8340: 3.0737389087677003
loss at step 8360: 3.158937060832977
loss at step 8380: 3.063744866847992
loss at step 8400: 3.063803267478943
validation loss: 3.210866157213847
validation accuracy: 10.929674457429048%
loss at step 8420: 3.16995393037796
loss at step 8440: 3.14454472064

loss at step 11920: 3.0050369024276735
loss at step 11940: 3.0233883261680603
loss at step 11960: 3.042835235595703
loss at step 11980: 3.0677093982696535
loss at step 12000: 3.0230388879776
validation loss: 3.129176624615987
validation accuracy: 11.675709515859767%
loss at step 12020: 3.055820894241333
loss at step 12040: 3.0321760416030883
loss at step 12060: 3.080505061149597
loss at step 12080: 2.967938709259033
loss at step 12100: 3.0617630958557127
loss at step 12120: 3.060823452472687
loss at step 12140: 3.043993330001831
loss at step 12160: 2.953952968120575
loss at step 12180: 3.0513290762901306
loss at step 12200: 3.071247625350952
loss at step 12220: 2.9987076997756956
loss at step 12240: 3.024428296089172
loss at step 12260: 3.0340832114219665
loss at step 12280: 3.039249229431152
loss at step 12300: 3.025560128688812
loss at step 12320: 2.966801941394806
loss at step 12340: 3.074874687194824
loss at step 12360: 2.9879834771156313
loss at step 12380: 3.0438493847846986
loss

loss at step 15780: 3.0605921387672423
loss at step 15800: 2.914723575115204
loss at step 15820: 2.9062338352203367
loss at step 15840: 3.02288738489151
loss at step 15860: 3.0138417363166807
loss at step 15880: 3.025317406654358
loss at step 15900: 3.053657019138336
loss at step 15920: 2.9703461170196532
loss at step 15940: 2.924605929851532
loss at step 15960: 2.962989556789398
loss at step 15980: 2.967585599422455
loss at step 16000: 2.9600739002227785
validation loss: 3.0751457357406617
validation accuracy: 12.087854757929883%
loss at step 16020: 2.9348230719566346
loss at step 16040: 2.980185353755951
loss at step 16060: 2.9786244869232177
loss at step 16080: 2.979057991504669
loss at step 16100: 2.991746699810028
loss at step 16120: 2.9444692492485047
loss at step 16140: 2.9237182974815368
loss at step 16160: 2.987958514690399
loss at step 16180: 2.978611671924591
loss at step 16200: 2.98181414604187
loss at step 16220: 2.967973160743713
loss at step 16240: 2.931735026836395
loss

...

loss at step 101220: 2.6691025853157044
loss at step 101240: 2.625395894050598
loss at step 101260: 2.5753268122673036
loss at step 101280: 2.6050233483314513
loss at step 101300: 2.6523296356201174
loss at step 101320: 2.6847665786743162
loss at step 101340: 2.6948291182518007
loss at step 101360: 2.6853113412857055
loss at step 101380: 2.759034752845764
loss at step 101400: 2.7054874420166017
loss at step 101420: 2.7358849167823793
loss at step 101440: 2.6349490761756895
loss at step 101460: 2.7603220343589783
loss at step 101480: 2.6653544425964357
loss at step 101500: 2.7170209527015685
loss at step 101520: 2.664554464817047
loss at step 101540: 2.6926305890083313
loss at step 101560: 2.637912356853485
loss at step 101580: 2.612354552745819
loss at step 101600: 2.7167787671089174
validation loss: 2.753519817988078
validation accuracy: 15.880634390651085%
loss at step 101620: 2.6247223615646362
loss at step 101640: 2.7726691365242004
loss at step 101660: 2.6700878500938416
loss at s

loss at step 105020: 2.6956525564193727
loss at step 105040: 2.7128366589546205
loss at step 105060: 2.680810809135437
loss at step 105080: 2.642867636680603
loss at step 105100: 2.643260860443115
loss at step 105120: 2.719448482990265
loss at step 105140: 2.6530309557914733
loss at step 105160: 2.657451891899109
loss at step 105180: 2.720752990245819
loss at step 105200: 2.6286710619926454
validation loss: 2.7473398129145306
validation accuracy: 15.588480801335558%
loss at step 105220: 2.6763653635978697
loss at step 105240: 2.54657701253891
loss at step 105260: 2.668469715118408
loss at step 105280: 2.633170962333679
loss at step 105300: 2.679142141342163
loss at step 105320: 2.6609315633773805
loss at step 105340: 2.6678587079048155
loss at step 105360: 2.617349588871002
loss at step 105380: 2.6016167521476747
loss at step 105400: 2.684893453121185
loss at step 105420: 2.635358726978302
loss at step 105440: 2.6918542861938475
loss at step 105460: 2.651079475879669
loss at step 10548

loss at step 108820: 2.6926472544670106
loss at step 108840: 2.618320369720459
loss at step 108860: 2.7179187536239624
loss at step 108880: 2.7108777165412903
loss at step 108900: 2.6983333587646485
loss at step 108920: 2.546515429019928
loss at step 108940: 2.7496787309646606
loss at step 108960: 2.8116978645324706
loss at step 108980: 2.688618516921997
loss at step 109000: 2.690885639190674
loss at step 109020: 2.682310688495636
loss at step 109040: 2.6846598863601683
loss at step 109060: 2.705125463008881
loss at step 109080: 2.701354277133942
loss at step 109100: 2.633997344970703
loss at step 109120: 2.6749712347984316
loss at step 109140: 2.682475709915161
loss at step 109160: 2.64698805809021
loss at step 109180: 2.570146417617798
loss at step 109200: 2.680081069469452
validation loss: 2.7673524030049643
validation accuracy: 15.44762103505843%
loss at step 109220: 2.677096796035767
loss at step 109240: 2.654175853729248
loss at step 109260: 2.6872895479202272
loss at step 109280

loss at step 112640: 2.6645275473594667
loss at step 112660: 2.691979467868805
loss at step 112680: 2.6795437693595887
loss at step 112700: 2.669618558883667
loss at step 112720: 2.6545138120651246
loss at step 112740: 2.7186007261276246
loss at step 112760: 2.6112676441669462
loss at step 112780: 2.667857086658478
loss at step 112800: 2.727476692199707
validation loss: 2.7420052909851074
validation accuracy: 15.661519198664442%
loss at step 112820: 2.6203613758087156
loss at step 112840: 2.6709481835365296
loss at step 112860: 2.647661340236664
loss at step 112880: 2.662482261657715
loss at step 112900: 2.720310890674591
loss at step 112920: 2.618266224861145
loss at step 112940: 2.6424999117851256
loss at step 112960: 2.6215763092041016
loss at step 112980: 2.8250272393226625
loss at step 113000: 2.6372488379478454
loss at step 113020: 2.621204102039337
loss at step 113040: 2.7044130206108092
loss at step 113060: 2.6996842861175536
loss at step 113080: 2.566091442108154
loss at step 

loss at step 116420: 2.756659984588623
loss at step 116440: 2.714279556274414
loss at step 116460: 2.7071182131767273
loss at step 116480: 2.664304792881012
loss at step 116500: 2.614995503425598
loss at step 116520: 2.551398479938507
loss at step 116540: 2.6691449165344237
loss at step 116560: 2.6349333763122558
loss at step 116580: 2.664230000972748
loss at step 116600: 2.6055002450942992
loss at step 116620: 2.640992558002472
loss at step 116640: 2.664368748664856
loss at step 116660: 2.6880857586860656
loss at step 116680: 2.6151037931442263
loss at step 116700: 2.625714099407196
loss at step 116720: 2.6638700008392333
loss at step 116740: 2.630499076843262
loss at step 116760: 2.6483338952064512
loss at step 116780: 2.6879353761672973
loss at step 116800: 2.651603376865387
validation loss: 2.7430141480763752
validation accuracy: 15.625%
loss at step 116820: 2.6489794492721557
loss at step 116840: 2.6270684123039247
loss at step 116860: 2.6639470219612122
loss at step 116880: 2.627

## Generating
This model has the Fast Wavenet Generation Algorithm (https://arxiv.org/abs/1611.09482) implemented. This might run faster on the cpu. You can give some starting data (of at least the length of receptive field) or let the model generate from zero. In my experience, a temperature between 0.5 and 1.0 yields the best results, but this may depend on the data set.

In [ ]:
start_data = data[250000][0] # use start data from the data set
start_data = torch.max(start_data, 0)[1] # convert one hot vectors to integers

def prog_callback(step, total_steps):
    print(str(100 * step // total_steps) + "% generated")

generated = model.generate_fast(num_samples=160000,
                                 first_samples=start_data,
                                 progress_callback=prog_callback,
                                 progress_interval=1000,
                                 temperature=1.0,
                                 regularize=0.)

In [ ]:
import IPython.display as ipd

ipd.Audio(generated, rate=16000)